In [28]:
import numpy as np
from tqdm import tqdm

In [75]:
class Intersection:
    def __init__(self, id):
        self.id = id
        self.street_in = {}
        self.street_out = {}
    
    def __repr__(self):
        return f"{self.id}: cars({sum(self.street_in.values())})"

In [30]:
class Street:
    def __init__(self, start, end, name, duration):
        self.start = start
        self.end = end
        self.name = name
        self.duration = duration

In [50]:
import math

class Car:
    def __init__(self, n_streets, streets):
        self.n_streets = n_streets
        self.streets = streets
        self.trip_duration = math.inf

In [32]:
class Problem:
    def __init__(self, duration, n_intersections, n_streets, n_cars, points, cars, streets, intersections):
        self.duration = duration
        self.n_intersections = n_intersections
        self.n_streets = n_streets
        self.n_cars = n_cars
        self.points = points
        self.cars = cars
        self.streets = streets
        self.intersections = intersections

    def __repr__(self):
        return f"T={self.duration};n_intersections={self.n_intersections};n_streets{self.n_streets};n_cars{self.n_cars};points{self.points}"

In [33]:
class Schedule:
    def __init__(self, intersection_id):
        self.intersection_id = intersection_id
        self.green_lights = list()

    def add_street(self, street_name, duration):
        self.green_lights.append({"name": street_name, "duration": duration})

    def __repr__(self):
        return f"{self.intersection_id}: {self.green_lights}"

In [68]:
def read_file(filename):
    # Read in content of file
    with open(filename) as f:
        lines = f.readlines()
    time, n_intersections, n_streets, n_cars, points = list(map(int, lines[0].strip().split()))
    streets = {}
    intersections = {} # steets arriving in a intersection
    for i in range(n_intersections):
        intersections[i] = Intersection(i)

    for i in range(n_streets):
        start, end, name, duration = lines[1 + i].strip().split()
        streets[name] = Street(int(start), int(end), name, int(duration))
        intersections[int(start)].street_out[name] = 0
        intersections[int(end)].street_in[name] = 0

    cars = []
    for i in range(n_cars):
        split = lines[1 + n_streets + i].strip().split()
        cars.append(Car(int(split[0]), split[1:]))
        
    return Problem(time, n_intersections, n_streets, n_cars, points, cars, streets, intersections) 

In [35]:
def generate_output(filename, schedules):
    with open(filename + '.out', 'w') as file:
        file.write(str(len(schedules)) + '\n')
        for schedule in schedules:
            file.write(str(schedule.intersection_id) + '\n')
            file.write(str(len(schedule.green_lights)) + '\n')
            for light in schedule.green_lights:
                file.write(f"{light['name']} {light['duration']}\n")

In [70]:
import os

for file in os.listdir('data'):
    if not file.endswith('.txt'): # or FILE.startswith('d'):
        continue
    problem = read_file(f"data/{file}")
    
    schedules = []
    for intersection_id, intersection in tqdm(problem.intersections.items()):
        schedule = Schedule(intersection_id)
        for street in intersection.street_in.keys():
            schedule.add_street(street, 1)
        schedules.append(schedule)

    # print(f"{file}: score = {score(deliveries)}")
    generate_output(f'data/{file}', schedules)

100%|██████████| 1662/1662 [00:00<00:00, 30473.09it/s]


In [146]:
import os

for file in os.listdir('data'):
    if not file.endswith('.txt'): # or FILE.startswith('d'):
        continue
        
    p = read_file(f"data/{file}")
    
    for car in p.cars:
        car.trip_duration = 0
        for s_name in car.streets:
            street = p.streets[s_name]
            car.trip_duration += street.duration
            p.intersections[street.end].street_in[s_name] += 1

    streets_star = {}
    for car in p.cars:
        i = p.streets[car.streets[0]].end
        if i not in streets_star:
            streets_star[i] = {}

        if car.streets[0] not in streets_star[i]:
            streets_star[i][car.streets[0]] = 0
        streets_star[i][car.streets[0]] += 1

    schedules = []
    for intersection_id, intersection in tqdm(p.intersections.items()):

        schedule = Schedule(intersection_id)
        if intersection_id in streets_star:
            for street, n_cars in streets_star[intersection_id].items():
                schedule.add_street(street, math.ceil(n_cars/5))
            for street, n_cars in intersection.street_in.items():
                if (n_cars > 0) and street not in streets_star[intersection_id]:
                    schedule.add_street(street, math.ceil(n_cars/5))
                    
        else:
            for street, n_cars in intersection.street_in.items():
                if (n_cars > 0):
                    schedule.add_street(street, math.ceil(n_cars/5))
        if len(schedule.green_lights) > 0:
            schedules.append(schedule)
            
                
#     for intersection_id, intersection in tqdm(p.intersections.items()):
#         schedule = Schedule(intersection_id)
#         for street, n_cars in intersection.street_in.items():
#             if (n_cars > 0):
#                 schedule.add_street(street, math.ceil(n_cars/5))
#         if len(schedule.green_lights) > 0:
#             schedules.append(schedule)

    # print(f"{file}: score = {score(deliveries)}")
    generate_output(f'data/{file}', schedules)

100%|██████████| 1662/1662 [00:00<00:00, 60796.56it/s]


In [139]:
p = read_file("data/e.txt")
    
    
for car in p.cars:
    car.trip_duration = 0
    for s_name in car.streets:
        street = p.streets[s_name]
        car.trip_duration += street.duration
        p.intersections[street.end].street_in[s_name] += 1

streets_star = {}
for car in p.cars:
    i = p.streets[car.streets[0]].end
    if i not in streets_star:
        streets_star[i] = {}
        
    if car.streets[0] not in streets_star[i]:
        streets_star[i][car.streets[0]] = 0
    streets_star[i][car.streets[0]] += 1

schedules = []
for intersection_id, intersection in tqdm(p.intersections.items()):
    
    schedule = Schedule(intersection_id)
    for street, n_cars in streets_star[intersection_id].items():
        schedule.add_street(street, n_cars)
    for street, n_cars in intersection.street_in.items():
        if (n_cars > 0) and street not in streets_star[intersection_id]:
            schedule.add_street(street, n_cars)
    if len(schedule.green_lights) > 0:
        schedules.append(schedule)

# print(f"{file}: score = {score(deliveries)}")
generate_output(f'data/e.txt', schedules)

100%|██████████| 500/500 [00:00<00:00, 125653.21it/s]


In [130]:
p = read_file("data/e.txt")
print(p)
        
for car in p.cars:
    car.trip_duration = 0
    for s_name in car.streets:
        street = p.streets[s_name]
        car.trip_duration += street.duration
        p.intersections[street.end].street_in[s_name] += 1

# p.cars.sort(key=lambda c: c.trip_duration, reverse=False)        

T=676;n_intersections=500;n_streets998;n_cars1000;points500


In [135]:
streets_star = set()

for car in p.cars:
    if p.streets[car.streets[0]].end == 499:
        streets_star.append(car.streets[0])
        #print(car.streets)
streets_star

['i-ejj',
 'be-ejj',
 'bha-ejj',
 'je-ejj',
 'dda-ejj',
 'dge-ejj',
 'cae-ejj',
 'gi-ejj',
 'dg-ejj',
 'dea-ejj',
 'ca-ejj',
 'cde-ejj',
 'bda-ejj',
 'bac-ejj',
 'cci-ejj',
 'i-ejj',
 'ig-ejj',
 'ejc-ejj',
 'fc-ejj',
 'ehi-ejj',
 'bag-ejj',
 'eda-ejj',
 'cdi-ejj',
 'i-ejj',
 'dia-ejj',
 'bie-ejj',
 'dhe-ejj',
 'bgi-ejj',
 'big-ejj',
 'egg-ejj',
 'dhe-ejj',
 'cgg-ejj',
 'dhg-ejj',
 'cfi-ejj',
 'cjg-ejj',
 'dha-ejj',
 'ba-ejj',
 'bdc-ejj',
 'cee-ejj',
 'eeg-ejj',
 'he-ejj',
 'efa-ejj',
 'cia-ejj',
 'cdi-ejj',
 'dde-ejj',
 'bde-ejj',
 'dja-ejj',
 'cfa-ejj',
 'bec-ejj',
 'ee-ejj',
 'ccg-ejj',
 'eac-ejj',
 'cje-ejj',
 'efe-ejj',
 'fi-ejj',
 'eag-ejj',
 'bbg-ejj',
 'cdi-ejj',
 'dji-ejj',
 'cc-ejj',
 'dde-ejj',
 'dbg-ejj',
 'bae-ejj',
 'ege-ejj',
 'he-ejj',
 'gi-ejj',
 'efc-ejj',
 'ba-ejj',
 'bei-ejj',
 'bgc-ejj',
 'eie-ejj',
 'ig-ejj',
 'bee-ejj',
 'he-ejj',
 'eii-ejj',
 'cdg-ejj',
 'ede-ejj',
 'eei-ejj',
 'dhc-ejj',
 'bbg-ejj',
 'daa-ejj',
 'dfg-ejj',
 'ji-ejj',
 'je-ejj',
 'bga-ejj',
 'ca-

In [89]:
schedules = []
for intersection_id, intersection in tqdm(p.intersections.items()):
    schedule = Schedule(intersection_id)
    for street, n_cars in intersection.street_in.items():
        if (n_cars > 0):
            schedule.add_street(street, n_cars)
    schedules.append(schedule)

100%|██████████| 4/4 [00:00<00:00, 1058.37it/s]


In [90]:
schedules

[0: [{'name': 'rue-de-londres', 'duration': 2}],
 1: [{'name': 'rue-d-amsterdam', 'duration': 1}, {'name': 'rue-d-athenes', 'duration': 1}],
 2: [{'name': 'rue-de-moscou', 'duration': 2}],
 3: [{'name': 'rue-de-rome', 'duration': 1}]]

In [49]:
p.streets['rue-de-londres'].start

2

In [41]:
p.cars[0].streets

['rue-de-londres', 'rue-d-amsterdam', 'rue-de-moscou', 'rue-de-rome']

In [38]:
p.intersections[0].street_in

{'rue-de-londres'}

In [15]:
i = Intersection(1)

In [30]:
i.street_in.append("test")
i.street_in

['test', 'test', 'test', 'test']

In [ ]:
import os

for file in os.listdir('data'):
    if not file.endswith('.txt'): # or FILE.startswith('d'):
        continue
    problem = read_file(f"data/{file}")
    
    schedules = []
    for intersection_id, intersection in tqdm(problem.intersections.items()):
        schedule = Schedule(intersection_id)
        for street in intersection.street_in:
            schedule.add_street(street, 1)
        schedules.append(schedule)

    # print(f"{file}: score = {score(deliveries)}")
    generate_output(f'data/{file}', schedules)